# Initial Mess Around

Want to make a three true outcomes leader board, with maybe a would it dong? cross over

In [1]:
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
from pybaseball import statcast_batter, spraychart

import pandas as pd

In [2]:
# this defaults to yesterday
df = statcast(start_dt='2022-06-04', end_dt='2022-06-04')

print(df.shape)

This is a large query, it may take a moment to complete


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.26it/s]

(4670, 92)


In [3]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1689,FF,2022-06-04,94.2,-1.3,5.8,"Wick, Rowan",405395,592858,force_out,hit_into_play,...,6,1,6,1,6,Infield shift,Standard,192,0.003,-0.499
1742,FF,2022-06-04,95.1,-1.45,5.66,"Wick, Rowan",405395,592858,NaN,ball,...,6,1,6,1,6,Infield shift,Standard,191,0.0,0.054
1768,FF,2022-06-04,94.9,-1.28,5.78,"Wick, Rowan",668800,592858,strikeout,swinging_strike,...,6,1,6,1,6,Standard,Standard,190,0.009,-0.385
1809,FF,2022-06-04,96.2,-1.32,5.68,"Wick, Rowan",668800,592858,NaN,foul,...,6,1,6,1,6,Standard,Standard,198,0.0,-0.13
1834,FF,2022-06-04,94.2,-1.47,5.71,"Wick, Rowan",668800,592858,NaN,ball,...,6,1,6,1,6,Standard,Standard,178,0.0,0.072


In [4]:
df.events.value_counts()

field_out                    486
strikeout                    287
single                       149
walk                         106
double                        51
home_run                      30
force_out                     27
grounded_into_double_play     18
hit_by_pitch                   7
field_error                    7
sac_fly                        6
caught_stealing_2b             4
triple                         3
fielders_choice_out            2
double_play                    2
fielders_choice                1
Name: events, dtype: int64

In [5]:
tto_events = df.loc[df["events"].isin(["strikeout", "walk", "home_run"])]

In [6]:
tto_events.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1768,FF,2022-06-04,94.9,-1.28,5.78,"Wick, Rowan",668800,592858,strikeout,swinging_strike,...,6,1,6,1,6,Standard,Standard,190,0.009,-0.385
2354,SL,2022-06-04,86.1,-1.69,5.57,"Wick, Rowan",660766,592858,strikeout,swinging_strike,...,6,1,6,1,6,Standard,Standard,110,0.007,-0.209
2546,KC,2022-06-04,77.3,-1.41,5.73,"Wick, Rowan",680977,592858,walk,ball,...,6,1,6,1,6,Standard,Standard,19,-0.007,0.321
3676,FF,2022-06-04,97.6,-1.37,5.7,"Wick, Rowan",502671,592858,strikeout,swinging_strike,...,5,1,5,1,5,Standard,Standard,195,0.007,-0.112
1972,FF,2022-06-04,91.5,-2.51,5.8,"Woodford, Jake",542364,663765,strikeout,swinging_strike,...,1,1,5,5,1,Standard,Standard,205,-0.002,-0.134


In [7]:
# ALSO NEED TO GET THE TOTAL OF PA FOR THE GAME!

outcome_counts = tto_events.groupby("batter")["events"].value_counts().unstack(fill_value=0)



In [8]:
outcome_counts.head()

events,home_run,strikeout,walk
batter,,,
405395,0,0,1
408234,0,2,0
425877,0,3,0
443558,0,0,1
444482,0,3,0


In [9]:
three_outcomes = outcome_counts.loc[(outcome_counts["home_run"] > 0) & (outcome_counts["strikeout"] > 0) & (outcome_counts["walk"] > 0)]

# Now try to calculate only three true outcomes

total_counts = df.groupby("batter")["events"].count()

three_outcomes = three_outcomes.merge(total_counts, on='batter', how='left')

three_outcomes

,home_run,strikeout,walk,events
batter,,,,
592273,1,1,1,5
669242,1,3,2,9


In [10]:
only_three_outcomes = three_outcomes.loc[three_outcomes[["home_run", "strikeout", "walk"]].sum(axis=1) == three_outcomes["events"]]

In [11]:
only_three_outcomes

,home_run,strikeout,walk,events
batter,,,,


In [12]:
player_ids = only_three_outcomes.index

player_names = playerid_reverse_lookup(player_ids, key_type='mlbam')

Gathering player lookup table. This may take a moment.


In [13]:
only_three_outcomes = only_three_outcomes.merge(player_names[["name_first", "name_last", "key_mlbam"]],
                                                left_on='batter',
                                                right_on='key_mlbam',
                                                how="left")

In [14]:
# double check this!

tto_events.loc[tto_events["batter"].isin(player_ids)]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp


In [15]:
# this is stupid but the plotting is based on team name, not abbreviation

# TODO: double check this! 
abbreviation_to_name = {
    "LAA" : "angels",
    "HOU" : "astros",
    "OAK" : "athletics",
    "TOR" : "blue_jays",
    "ATL" : "braves",
    "MIL" : "brewers",
    "STL" : "cardinals",
    "CHC" : "cubs",
    "ARI" : "diamondbacks",
    "LAD" : "dodgers",
    "SF" : "giants",
    "CLE" : "indians",
    "TOR" : "mariners",
    "MIA" : "marlins",
    "NYM" : "mets",
    "WAS" : "nationals",
    "BAL" : "orioles",
    "SD" : "padres",
    "PHL" : "phillies",
    "PIT" : "pirates",
    "TEX" : "rangers",
    "TBR" : "rays",
    "BOS" : "red_sox",
    "CIN" : "reds",
    "COL" : "rockies",
    "KC" : "royals",
    "DET" : "tigers",
    "MIN" : "twins",
    "CWS" : "white_sox",
    "NYY" : "yankees",
    "else" : "generic"
}


In [16]:
# check if there are none!

for row in only_three_outcomes.iterrows():
    actual_row = row[1]
    
    player_name = actual_row.name_first.capitalize() + " " + actual_row.name_last.capitalize()
        
    sub_data = tto_events.loc[tto_events["batter"] == actual_row.key_mlbam]
    
    # TODO: put all of the things in the title
    title = player_name + " " + "HR(s): " + str(actual_row.home_run) +\
    ", " + "BB(s): " + str(actual_row.walk) + ", and K(s): " + str(actual_row.strikeout)
    
    
    # figure out how to save this as png
    print(spraychart(sub_data, abbreviation_to_name[sub_data.home_team.iloc[0]], title = title))

In [20]:
# this is for gut checks

id_to_check = 669242

tto_events.loc[tto_events["batter"] == id_to_check]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2763,SL,2022-06-04,80.1,-4.37,3.44,"Effross, Scott",669242,664123,strikeout,called_strike,...,5,1,5,1,5,Infield shift,Standard,82,0.052,-0.97
604,SL,2022-06-04,81.7,-1.21,6.72,"Swarmer, Matt",669242,664161,home_run,hit_into_play,...,3,1,3,1,3,Standard,Standard,102,-0.076,1.0
1793,SL,2022-06-04,86.4,-2.82,4.6,"Givens, Mychal",669242,571710,walk,ball,...,3,3,3,3,3,Infield shift,Standard,128,-0.034,0.19
296,FC,2022-06-04,90.4,-1.54,5.72,"Kilian, Caleb",669242,668873,strikeout,called_strike,...,1,3,1,3,1,Infield shift,Standard,183,0.018,-0.158
1827,SI,2022-06-04,91.7,-1.3,5.67,"Kilian, Caleb",669242,668873,walk,ball,...,1,0,1,0,1,Standard,Standard,202,-0.047,0.168
4263,FF,2022-06-04,96.4,-1.58,5.61,"Kilian, Caleb",669242,668873,strikeout,swinging_strike,...,0,0,0,0,0,Infield shift,Standard,198,0.022,-0.141


In [21]:
total_counts[id_to_check]

9

In [25]:
df[df["batter"] == id_to_check].game_pk

2763    663179
2777    663179
2871    663179
2962    663179
3122    663179
3174    663179
604     663179
1631    663179
1662    663179
1685    663179
1714    663179
4537    663179
1821    663152
1865    663152
1912    663152
1955    663152
1793    663152
1868    663152
1896    663152
1956    663152
1969    663152
2019    663152
296     663152
338     663152
365     663152
374     663152
1827    663152
1856    663152
1876    663152
1946    663152
4263    663152
4459    663152
4663    663152
Name: game_pk, dtype: Int64